In [ ]:
%matplotlib inline
import numpy as np
import sys
import tensorflow as tf
import copy
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
# depending on the classification model use, we might need to import other packages
#from sklearn import svm
#from sklearn.ensemble import RandomForestClassifier
import sklearn
from matplotlib import pyplot as plt
import pickle as pkl

from datasets import DatasetUCI
from envs import LalEnvTargetAccuracy

from estimator import Estimator
from helpers import Minibatch, ReplayBuffer
from dqn import DQN

In [ ]:
from Test_AL import check_performance, check_performance_for_figure

Strategies to test

In [ ]:
# Random sampling
rs = True
# Uncertainty sampling
us = True
# LAL-RL learnt strategy on other datasets
rl = True
# LAL-RL learnt strategy on the same dataset (another half)
rl_notransfer = False
# LAL-independent and LAL-iterative strategies
lal = False
# ALBE strategy that learns a combination of rs, us and quire
albe = False
# QUIRE strategy
quire = False

## Experiment parameters

In [ ]:
DIRNAME_TRANSFER = './agents/1-australian-logreg-8-to-1/'
DIRNAME_NOTRANSFER = ''
DIRNAME_RESULTS = './AL_results/test-agent-australian.p'

TOLERANCE_LEVEL = 0.98
test_dataset_names = ['australian']

N_STATE_ESTIMATION = 30
SUBSET = -1 # -1 for using all datapoints, 0 for even, 1 for odd
SIZE = 100

N_JOBS = 1 # can set more if we want to parallelise
QUALITY_METHOD = metrics.accuracy_score

N_EXPERIMENTS = 500

Can use different models for classifier
<br>
`LogisticRegression(n_jobs=N_JOBS)` <br>
SVM: <br>
`svm.SVC(probability=True)` <br>
RF: <br>
`RandomForestClassifier(50, oob_score=True, n_jobs=N_JOBS)`

In [ ]:
dataset = DatasetUCI(possible_names=test_dataset_names, n_state_estimation=N_STATE_ESTIMATION, subset=SUBSET, size=SIZE)
model = LogisticRegression(n_jobs=N_JOBS)
env = LalEnvTargetAccuracy(dataset, model, quality_method=QUALITY_METHOD, tolerance_level=TOLERANCE_LEVEL)

## Prepare AL methods

#### Methods for random sampling and uncertainty sampling

In [ ]:
if rs:
    from Test_AL import policy_random
if us:
    from Test_AL import policy_uncertainty

#### Load RL model

In [ ]:
if rl:
    from Test_AL import policy_rl
    tf.reset_default_graph()
    # Load the DQN agent from DIRNAME_TRANSFER
    agent = DQN(experiment_dir=DIRNAME_TRANSFER,
            observation_length=N_STATE_ESTIMATION,
            learning_rate=1e-3,
            batch_size=32,
            target_copy_factor=0.01,
            bias_average=0,
           )

#### Load RL model with no transfer

In [ ]:
if rl_notransfer:
    from Test_AL import policy_rl
    tf.reset_default_graph()
    # Load the DQN agent from DIRNAME_NOTRANSFER
    agent = DQN(experiment_dir=DIRNAME_NOTRANSFER,
            observation_length=N_STATE_ESTIMATION,
            learning_rate=1e-3,
            batch_size=32,
            target_copy_factor=0.01,
            bias_average=0,
           )

#### Load LAL models

In [ ]:
if lal:
    from sklearn.ensemble import RandomForestRegressor
    from Test_AL import policy_LAL
    
    # LAL-INDEPENDENT
    # parameters
    fn = 'LAL-randomtree-simulatedunbalanced-big.npz'
    parameters = {'est': 2000, 'depth': 40, 'feat': 6 }
    # load data
    filename = '../LALfiles/'+fn
    regression_data = np.load(filename)
    regression_features = regression_data['arr_0']
    regression_labels = regression_data['arr_1']
    # build model
    print('building lal model..')
    lal_model1 = RandomForestRegressor(n_estimators = parameters['est'], max_depth = parameters['depth'], 
                                     max_features=parameters['feat'], oob_score=True, n_jobs=8)
    lal_model1.fit(regression_features, np.ravel(regression_labels))    
    print('the model is built!')
    print('oob score = ', lal_model1.oob_score_)

    # LAL-ITERATIVE
    # parameters
    fn = 'LAL-iterativetree-simulatedunbalanced-big.npz'
    parameters = {'est': 1000, 'depth': 40, 'feat': 6 }
    # load data
    filename = '../LALfiles/'+fn
    regression_data = np.load(filename)
    regression_features = regression_data['arr_0']
    regression_labels = regression_data['arr_1']
    # build model
    print('building lal model..')
    lal_model2 = RandomForestRegressor(n_estimators = parameters['est'], max_depth = parameters['depth'], 
                                     max_features=parameters['feat'], oob_score=True, n_jobs=8)
    lal_model2.fit(regression_features, np.ravel(regression_labels))    
    print('the model is built!')
    print('oob score = ', lal_model2.oob_score_)

#### Prepare for ALBE

Depending on the classifier, need to use different adapter <br>
`SklearnProbaAdapter(LogisticRegression(n_jobs=N_JOBS))` for logistic regression or <br>
`SklearnProbaAdapter(svm.SVC(probability=True))` for SVM or <br>
`SklearnProbaAdapter(RandomForestClassifier(50, n_jobs=1))` for RF

In [ ]:
if albe:
    import sys
    sys.path.append('./libact/')
    from libact.base.dataset import Dataset
    from libact.query_strategies import QUIRE, UncertaintySampling, ActiveLearningByLearning
    from libact.models import SklearnProbaAdapter
    from Test_AL import policy_ALBE
    
    def reset_albe(dataset, env):
        """Initialises libact to perform ALBE"""
        adapter = SklearnProbaAdapter(LogisticRegression(n_jobs=1)) 
        nolabels = np.array(([None] * len(dataset.train_labels)))
        libactlabels = nolabels
        libactlabels[env.indeces_known] = dataset.train_labels[env.indeces_known]
        trn_ds = Dataset(dataset.train_data, libactlabels)
        # max number of iterations is needed here
        qs = ActiveLearningByLearning(trn_ds, query_strategies=[UncertaintySampling(trn_ds, model=adapter), QUIRE(trn_ds)], T=1000, uniform_sampler=True, model=adapter)
        #qs = QUIRE(trn_ds, model=adapter)
        #qs = ActiveLearningByLearning(trn_ds, query_strategies=[QUIRE(trn_ds)], T=100, uniform_sampler=True, model=adapter)
        return qs, trn_ds

#### Prepare for QUIRE

In [ ]:
if quire:
    import sys
    sys.path.append('./libact/')
    from libact.base.dataset import Dataset
    from libact.query_strategies import QUIRE
    from libact.models import SklearnProbaAdapter
    from Test_AL import policy_QUIRE
    
    def reset_quire(dataset, env):
        """"Initialises libact to perform QUIRE"""
        adapter = SklearnProbaAdapter(LogisticRegression(n_jobs=1))
        nolabels = np.array(([None] * len(dataset.train_labels)))
        libactlabels = nolabels
        libactlabels[env.indeces_known] = dataset.train_labels[env.indeces_known]
        trn_ds = Dataset(dataset.train_data, libactlabels)
        # max number of iterations is needed here
        qs = QUIRE(trn_ds, model=adapter)
        return qs, trn_ds

## Run the experiemnts

In [ ]:
# Results will be stored in all_results dictionary
all_results = {}
all_scores_rand = []
all_scores_uncert = []
all_scores_rl = []
all_scores_rl_notransfer = []
all_scores_LAL_independant = []
all_scores_LAL_iterative = []
all_scores_ALBE = []
all_scores_QUIRE = []

for experiment in range(N_EXPERIMENTS):
    print(experiment, end=' ')
    # reset the environment
    state, next_action_state = env.reset()
    # run the experiments
    # 1. copy the initial state and environment 
    # so that all strategies start from the same point            
    # 2. done variable indicates when terminal state is reached
    # 3. repeat until terminal state is reached
    # 4. select an action according to the policy
    # to see the prob of selected action: taken_action_state = next_action_state_uncert[:,action]
    # 5. make a step in the environment
    # 6. keep track of the scores in the episode
    if rs:
        env_rand = copy.deepcopy(env)
        state_rand = copy.deepcopy(state)
        done = False
        while not(done):
            action = policy_random(env_rand.n_actions)
            _, _, _, done = env_rand.step(action)
        all_scores_rand.append(env_rand.episode_qualities)
    if us:
        next_action_state_uncert = next_action_state
        env_uncert = copy.deepcopy(env)
        state_uncert = copy.deepcopy(state)
        done = False
        while not(done):
            action = policy_uncertainty(next_action_state_uncert[0,:])
            next_state, next_action_state_uncert, reward, done = env_uncert.step(action)
        all_scores_uncert.append(env_uncert.episode_qualities)
    if rl:
        next_action_state_rl = next_action_state
        env_rl = copy.deepcopy(env)
        state_rl = copy.deepcopy(state)
        done = False
        while not(done):
            action = policy_rl(agent, state_rl, next_action_state_rl)        
            next_state, next_action_state_rl, reward, done = env_rl.step(action)
            state_rl = next_state
        all_scores_rl.append(env_rl.episode_qualities)
    if rl_notransfer:
        next_action_state_rl_notransfer = next_action_state
        env_rl_notransfer = copy.deepcopy(env)
        state_rl_notransfer = copy.deepcopy(state)
        done = False
        while not(done):
            action = policy_rl(agent_notransfer, state_rl_notransfer, next_action_state_rl_notransfer)        
            next_state, next_action_state_rl_notransfer, reward, done = env_rl_notransfer.step(action)
            state_rl_notransfer = next_state
        all_scores_rl_notransfer.append(env_rl_notransfer.episode_qualities)
    if lal:
        next_action_state_LAL_independant = next_action_state
        env_LAL_independant = copy.deepcopy(env)
        state_LAL_independant = copy.deepcopy(state)
        done = False
        while not(done):
            env_LAL_independant.for_lal()
            action = policy_LAL(dataset, env_LAL_independant, lal_model1)
            next_state, next_action_state_LAL_independant, reward, done = env_LAL_independant.step(action)
        all_scores_LAL_independant.append(env_LAL_independant.episode_qualities)
        next_action_state_LAL_iterative = next_action_state
        env_LAL_iterative = copy.deepcopy(env)
        state_LAL_iterative = copy.deepcopy(state)
        done = False
        while not(done):
            env_LAL_iterative.for_lal()
            action = policy_LAL(dataset, env_LAL_iterative, lal_model2)
            next_state, next_action_state_LAL_iterative, reward, done = env_LAL_iterative.step(action)
        all_scores_LAL_iterative.append(env_LAL_iterative.episode_qualities)
    if albe:
        next_action_state_ALBE = next_action_state
        env_ALBE = copy.deepcopy(env)
        state_ALBE = copy.deepcopy(state)
        qs, trn_ds = reset_albe(dataset, env_ALBE)
        done = False
        while not(done):
            action = policy_ALBE(qs, trn_ds, env_ALBE, dataset)
            next_state, next_action_state_ALBE, reward, done = env_ALBE.step(action)
        all_scores_ALBE.append(env_ALBE.episode_qualities)
    if quire:
        next_action_state_QUIRE = next_action_state
        env_QUIRE = copy.deepcopy(env)
        state_QUIRE = copy.deepcopy(state)
        qs, trn_ds = reset_quire(dataset, env_QUIRE)
        done = False
        while not(done):
            action = policy_QUIRE(qs, trn_ds, env_QUIRE, dataset)
            next_state, next_action_state_QUIRE, reward, done = env_QUIRE.step(action)
        all_scores_QUIRE.append(env_QUIRE.episode_qualities)
# record the results
all_results['all_scores_rand'] = all_scores_rand
all_results['all_scores_uncert'] = all_scores_uncert
all_results['all_scores_rl'] = all_scores_rl
all_results['all_scores_rl_notransfer'] = all_scores_rl_notransfer
all_results['all_scores_LAL_independant'] = all_scores_LAL_independant
all_results['all_scores_LAL_iterative'] = all_scores_LAL_iterative
all_results['all_scores_ALBE'] = all_scores_ALBE
all_results['all_scores_QUIRE'] = all_scores_QUIRE
pkl.dump(all_results, open(DIRNAME_RESULTS, "wb" ))

## Load the results

In [ ]:
all_results = pkl.load(open(DIRNAME_RESULTS, "rb" ) )

In [ ]:
all_scores_rand = all_results['all_scores_rand']
all_scores_uncert = all_results['all_scores_uncert']
all_scores_rl = all_results['all_scores_rl']
all_scores_rl_notransfer = all_results['all_scores_rl_notransfer']
all_scores_LAL_independant = all_results['all_scores_LAL_independant']
all_scores_LAL_iterative = all_results['all_scores_LAL_iterative']
all_scores_ALBE = all_results['all_scores_ALBE']
all_scores_QUIRE = all_results['all_scores_QUIRE']

## Check the results

#### Compute the mean duration, it's std, median and max

In [ ]:
max_duration = 0
if rs:
    print("Random")
    all_scores_rand, all_durations_rand = check_performance(all_scores_rand)
    max_duration = max(max_duration, max(all_durations_rand))
if us:
    print("Uncertainty")
    all_scores_uncert, all_durations_uncert = check_performance(all_scores_uncert)
    max_duration = max(max_duration, max(all_durations_uncert))
if rl:
    print("RL")
    all_scores_rl, all_durations_rl = check_performance(all_scores_rl)
    max_duration = max(max_duration, max(all_durations_rl))
if rl_notransfer:
    print("RL without transfer")
    all_scores_rl_notransfer, all_durations_rl_notransfer = check_performance(all_scores_rl_notransfer)
    max_duration = max(max_duration, max(all_durations_rl_notransfer))
if lal:
    print("LAL independant")
    all_scores_LAL_independant, all_durations_LAL_independant = check_performance(all_scores_LAL_independant)
    max_duration = max(max_duration, max(all_durations_LAL_independant))
    print("LAL iterative")
    all_scores_LAL_iterative, all_durations_LAL_iterative = check_performance(all_scores_LAL_iterative)
    max_duration = max(max_duration, max(all_durations_LAL_iterative))
if albe:
    print("ALBE")
    all_scores_ALBE, all_durations_ALBE = check_performance(all_scores_ALBE)
    max_duration = max(max_duration, max(all_durations_ALBE))
if quire:
    print("QUIRE")
    all_scores_QUIRE, all_durations_QUIRE = check_performance(all_scores_QUIRE)
    max_duration = max(max_duration, max(all_durations_QUIRE))

#### Compute the relative scores that can be used to plot the results

In [ ]:
scores_relative_rand = check_performance_for_figure(all_scores_rand, max_duration)
scores_relative_uncert = check_performance_for_figure(all_scores_uncert, max_duration)
scores_relative_rl = check_performance_for_figure(all_scores_rl, max_duration)
scores_relative_rl_notransfer = check_performance_for_figure(all_scores_rl_notransfer, max_duration)
scores_relative_LAL_independant = check_performance_for_figure(all_scores_LAL_independant, max_duration)
scores_relative_LAL_iterative = check_performance_for_figure(all_scores_LAL_iterative, max_duration)
scores_relative_ALBE = check_performance_for_figure(all_scores_ALBE, max_duration)
scores_relative_QUIRE = check_performance_for_figure(all_scores_QUIRE, max_duration)

#### Plot the results

In [ ]:
plt.figure(figsize=(20,10))
if rs:
    m_line = np.mean(scores_relative_rand, axis=0)
    var_line = np.var(scores_relative_rand, axis=0)
    plt.plot(m_line, linewidth=2.0, label = 'random', color='k')
    plt.fill_between(range(np.size(m_line)), m_line - var_line, m_line + var_line, color='k', alpha=0.2)
if us:
    m_line = np.mean(scores_relative_uncert, axis=0)
    var_line = np.var(scores_relative_uncert, axis=0)
    plt.plot(m_line, linewidth=2.0, label = 'uncertainty', color='b')
    plt.fill_between(range(np.size(m_line)), m_line - var_line, m_line + var_line, color='b', alpha=0.2)
if rl:
    m_line = np.mean(scores_relative_rl, axis=0)
    var_line = np.var(scores_relative_rl, axis=0)
    plt.plot(m_line, linewidth=2.0, label = 'rl', color='red')
    plt.fill_between(range(np.size(m_line)), m_line - var_line, m_line + var_line, color='red', alpha=0.2)
if rl_notransfer:
    m_line = np.mean(scores_relative_rl_notransfer, axis=0)
    var_line = np.var(scores_relative_rl_notransfer, axis=0)
    plt.plot(m_line, linewidth=2.0, label = 'rl no transfer', color='red')
    plt.fill_between(range(np.size(m_line)), m_line - var_line, m_line + var_line, color='red', alpha=0.2)
if lal:
    m_line = np.mean(scores_relative_LAL_independant, axis=0)
    var_line = np.var(scores_relative_LAL_independant, axis=0)
    plt.plot(m_line, linewidth=2.0, label = 'LAL-independant', color='c')
    plt.fill_between(range(np.size(m_line)), m_line - var_line, m_line + var_line, color='c', alpha=0.2)    
    m_line = np.mean(scores_relative_LAL_iterative, axis=0)
    var_line = np.var(scores_relative_LAL_iterative, axis=0)
    plt.plot(m_line, linewidth=2.0, label = 'LAL-iterative', color='m')
    plt.fill_between(range(np.size(m_line)), m_line - var_line, m_line + var_line, color='m', alpha=0.2)
if albe:
    m_line = np.mean(scores_relative_ALBE, axis=0)
    var_line = np.var(scores_relative_ALBE, axis=0)
    plt.plot(m_line, linewidth=2.0, label = 'ALBE', color='g')
    plt.fill_between(range(np.size(m_line)), m_line - var_line, m_line + var_line, color='g', alpha=0.2)
if quire:
    m_line = np.mean(scores_relative_QUIRE, axis=0)
    var_line = np.var(scores_relative_QUIRE, axis=0)
    plt.plot(m_line, linewidth=2.0, label = 'QUIRE', color='y')
    plt.fill_between(range(np.size(m_line)), m_line - var_line, m_line + var_line, color='y', alpha=0.2)
plt.legend()